# Address Segmentation
Conversion of address points into segmented address ranges along a road network.

### Load and preview dataframes

In [9]:
import geopandas as gpd
addresses = gpd.read_file("C:/scratch/City_Of_Yellowknife.gpkg", layer="addresses")
roads = gpd.read_file("C:/scratch/City_Of_Yellowknife.gpkg", layer="roads")

In [11]:
addresses.head()

,number,street,geometry
0,43A,OTTO DRIVE,POINT Z (-114.34198 62.47255 0.00000)
1,48,TAYLOR ROAD,POINT Z (-114.38994 62.44161 0.00000)
2,136B,DE WEERDT DRIVE,POINT Z (-114.37009 62.46390 0.00000)
3,129B,HAENER DRIVE,POINT Z (-114.37518 62.46337 0.00000)
4,2B,STIRLING COURT,POINT Z (-114.37021 62.46631 0.00000)


In [10]:
roads.head()

,FEATURECOD,COMMUNITY,RTENAME2EN,SURFACE,DC_ENDDATE,ROADNAME,geometry
0,RSTRLIN,Northwest Territories,None,Unpaved,2015-09-28,VEE LAKE ROAD,"LINESTRING (-114.36956 62.54188, -114.36956 62..."
1,RRESLIN,Northwest Territories,None,Unpaved,2015-09-28,None,"LINESTRING (-114.36646 62.54697, -114.36646 62..."
2,RHWYLIN,Northwest Territories,Highway 4,Paved,2015-09-28,INGRAHAM TRAIL,"LINESTRING (-114.30128 62.50890, -114.30103 62..."
3,RARTLIN,Northwest Territories,None,Unpaved,2015-09-28,DETTAH ROAD,"LINESTRING (-114.30082 62.47320, -114.30080 62..."
4,RSTRLIN,Northwest Territories,None,Unpaved,2015-09-28,DETTAH,"LINESTRING (-114.30939 62.41112, -114.30967 62..."
